In [12]:
import pandas as pd
from collections import Counter
from itertools import combinations

In [13]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1704,tt0462538,75000000,527068851,The Simpsons Movie,Dan Castellaneta|Julie Kavner|Nancy Cartwright...,David Silverman,See our family. And feel better about yours.,After Homer accidentally pollutes the town's w...,87,Animation|Comedy|Family,Gracie Films|Twentieth Century Fox Film Corpor...,7/25/2007,6.9,2007
1538,tt0452624,32000000,5914908,The Good German,George Clooney|Cate Blanchett|Tobey Maguire|To...,Steven Soderbergh,"If war is Hell, then what comes after?",An American journalist played by George Cloone...,108,War|Crime|Drama|Mystery|Romance,Warner Bros.|Section Eight Productions|Sunset ...,12/8/2006,5.8,2006
677,tt0482606,9000000,82391145,The Strangers,Liv Tyler|Scott Speedman|Alex Fisher|Peter Cla...,Bryan Bertino,Lock the door. Pretend you're safe.,"After returning from a wedding reception, a co...",90,Thriller|Horror|Mystery,Rogue Pictures|Vertigo Entertainment,5/29/2008,5.9,2008
636,tt0455824,130000000,49554002,Australia,Nicole Kidman|Hugh Jackman|Essie Davis|David W...,Baz Luhrmann,Welcome to Australia!,"Set in northern Australia before World War II,...",165,Drama,Bazmark Films|Ingenious Film Partners|Twentiet...,11/18/2008,6.3,2008
1001,tt1259521,30000000,66486080,The Cabin in the Woods,Kristen Connolly|Chris Hemsworth|Anna Hutchiso...,Drew Goddard,If you hear a strange sound outside... have sex,Five college friends spend the weekend at a re...,95,Horror|Thriller,Lionsgate|Mutant Enemy Productions,4/12/2012,6.5,2012


In [14]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
imdb_id                 1889 non-null object
budget                  1889 non-null int64
revenue                 1889 non-null int64
original_title          1889 non-null object
cast                    1889 non-null object
director                1889 non-null object
tagline                 1889 non-null object
overview                1889 non-null object
runtime                 1889 non-null int64
genres                  1889 non-null object
production_companies    1889 non-null object
release_date            1889 non-null object
vote_average            1889 non-null float64
release_year            1889 non-null int64
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [16]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:
# Переведем колонку release_date из формата object в формат datetime64
data['release_date'] = pd.to_datetime(data['release_date'])

In [17]:
# для задания 21 понадобится получить месяц из числа, а также колонка месяца
import calendar
data['release_month'] = pd.DatetimeIndex(data['release_date']).month

In [18]:
# Создадим колонку прибыльности фильма из разницы между сборами и бюджетом
data['profit'] = data['revenue'] - data['budget']

In [19]:
# получение имени сезона по номеру месяца
def month_to_season(month):
    if (month>11 or month<3):
        return "winter"
    elif (month>8):
        return "autumn"
    elif (month>5):
        return "summer"
    
    return "spring"

In [20]:
# В ответах часто требуется сделать строку состоящую из имени и кода на imdb.
# Чтобы не писать одинаковые крупные конструкции сделаем короткую функцию
def answer_string(single_string_dataframe):
    # если вход пустой, то возвращаем пустую строку
    if (len(single_string_dataframe) == 0):
        return ''
    dataframe_string = single_string_dataframe.iloc[0]
    return dataframe_string['original_title'] + ' (' + dataframe_string['imdb_id'] + ')'

In [21]:
# В ответах часто требуются округленные числа уже в integer типе.
# round не подходит так как округлив мы все равно имеем десятичное (например, 110.0).
# Поэтому напишем функцию
def int_round(input_float):
    return int(round(input_float))

In [22]:
# функция для выбора ответа на вопрос 27. Вызывается один раз, но громоздка и 
# не относится к сути задания, поэтому вынесена отдельно
# на вход варианты ответы из теста, варианты ответа полученные мной и
# разделитель вариантов ответов на теста на актеров
def check_correct_answer27(test_answer_variants, answer_variants_list, sepp):
    
    # список листов ответов разделенных по фильмам
    test_answer_variants_lists = []

    # заполняем список листов ответов разделенных по фильмам
    for i in range(0, len(test_answer_variants)):
        test_answer_variants_lists.append(test_answer_variants[i].split(sepp))
    
    have_answer = False
    one_answer = ''
    for kk in range(0, len(answer_variants_list)):
        for i in range(0, len(test_answer_variants_lists)):
            for j in range(0, len(test_answer_variants_lists[i])):
                if test_answer_variants_lists[i][j] not in answer_variants_list[kk]:
                    break
            else:
                if (have_answer == True):
                    print("Ошибка - два правильных ответа")
                    # генерируем ошибку
                    tt = 2/0
                else:
                    have_answer = True
                    one_answer = test_answer_variants[i]
    return one_answer

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [23]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [24]:
# +
# тут пишем ваш код для решения данного вопроса:
# 1. получение строки из базы ответом
res_maxb = data[data.budget == data.budget.max()]#[['original_title', 'imdb_id']]
# 2. склеивание их в нужную строку ответа
answers['1'] = answer_string(res_maxb)
# далее при односложных решениях шаги 1 и 2 будут объединены
print(answers['1'])

Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [25]:
# +
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = answer_string(data[data.runtime == data.runtime.max()])
print(answers['2'])

Gods and Generals (tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [26]:
# +
answers['3'] = answer_string(data[data.runtime == data.runtime.min()])
print(answers['3'])

Winnie the Pooh (tt1449283)


# 4. Какова средняя длительность фильмов?


In [27]:
# +
answers['4'] = int_round(data.describe().runtime['mean'])
print(answers['4'])

110


# 5. Каково медианное значение длительности фильмов? 

In [28]:
# +
# медиана - это 50%-й квантиль
answers['5'] = int_round(data.describe().runtime['50%'])
print(answers['5'])

107


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [29]:
# +
answers['6'] = answer_string(data[data.profit == data.profit.max()])
print(answers['6'])

Avatar (tt0499549)


# 7. Какой фильм самый убыточный? 

In [30]:
# +
answers['7'] = answer_string(data[data.profit == data.profit.min()])
print(answers['7'])

The Lone Ranger (tt1210819)


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [31]:
# +
answers['8'] = len(data[data.profit >= 0])
print(answers['8'])

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [32]:
# +
# вначале получим фильмы за 2008 год
year_2008_df = data[data.release_year == 2008]
# теперь выберем тот, что с максимальной прибылью
answers['9'] = answer_string(year_2008_df[year_2008_df.profit == year_2008_df.profit.max()])
print(answers['9'])

The Dark Knight (tt0468569)


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [33]:
# +
# вначале получим фильмы за 2012-2014 года
years_12_to_14_df = data[(data.release_year <= 2014) & (data.release_year >= 2012)]
# теперь выберем тот, что с максимальной убыточностью
answers['10'] = answer_string(years_12_to_14_df[years_12_to_14_df.profit == years_12_to_14_df.profit.min()])
print(answers['10'])

The Lone Ranger (tt1210819)


# 11. Какого жанра фильмов больше всего?

In [34]:
# +
# тут будет код на сразу пять заданий (11, 12, 13, 14, 17), подготовка данных для ответов

# создадим счетчики и на жанры фильмов, на жанры прибыльных фильмов, на кассовых режиссеров
# и словари на счетчики жанров для режиссеров и актеров
genres = Counter()
genres_profitable = Counter()
directors = Counter()
directors_genres = {}

actors_genres = {}

for row in range(0, len(data)):
    # получим строку, чтобы не удлинять код каждый раз
    next_row_data = data.iloc[row]
    
    # идем по всем строкам и добавляем жанры (предварительно разделив строку) в локальный счетчик
    local_counter = Counter()
    for one_genre in next_row_data.genres.split('|'):
        local_counter[one_genre] += 1
        
    # далее прибавляем локальный к счетчику жанров всегда, 
    # а к счетчику прибыльных только если фильм прибыльный
    genres += local_counter
    if (next_row_data.profit > 0):
        genres_profitable += local_counter

    for one_dir in next_row_data.director.split('|'):
        directors[one_dir] += next_row_data.revenue
        # получаем текущий счетчик жанров дле режиссера или по умолчанию пустой
        one_dir_genres = directors_genres.get(one_dir, Counter())
        one_dir_genres += local_counter
        # сохраняем счетчик жанров режиссера
        directors_genres[one_dir] = one_dir_genres
        
    for one_actor in next_row_data.cast.split('|'):
        # получаем текущий счетчик жанров дле актера или по умолчанию пустой
        one_actor_genres = actors_genres.get(one_actor, Counter())
        one_actor_genres += local_counter
        # сохраняем счетчик жанров актера
        actors_genres[one_actor] = one_actor_genres

In [35]:
# получаем максимальное значение счетчика жанров
answers['11'] = genres.most_common(1)[0][0]
print(answers['11'])

Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [36]:
# +
# получаем максимальное значение счетчика самых прибыльных жанров
answers['12'] = genres_profitable.most_common(1)[0][0]
print(answers['12'])

Drama


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [37]:
# +
# получаем максимальное значение счетчика самых кассовых режиссеров
answers['13'] = directors.most_common(1)[0][0]
print(answers['13'])

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [38]:
# +
# получим отсортированный по количеству Action фильмов словарь на счетчики жанров для режиссеров
#лямбда возвращает счетчик фильмов жанраа Action если такие были, иначе 0
directors_genres_sorted = sorted(directors_genres.items(), key=lambda i: i[1].get('Action', 0), reverse = True)
# получим имя режиссера
answers['14'] = directors_genres_sorted[0][0]
print(answers['14'])

Robert Rodriguez


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [39]:
# +
# получим фильмы 2012 года
year_2012 = data[data.release_year == 2012]

# посчитаем кассовые сборы фильмов с каждым актером
actors_revenue = Counter()
for row in range(0, len(year_2012)):
    next_row_data = year_2012.iloc[row]
    for one_actor in next_row_data.cast.split('|'):
        actors_revenue[one_actor] += next_row_data.revenue

# получаем максимальное значение счетчика
answers['15'] = actors_revenue.most_common(1)[0][0]
print(answers['15'])

Chris Hemsworth


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [40]:
# +
# получим высокобюджетные фильмы
higher_mean = data[data.budget > data.describe().budget['mean']]

# посчитаем количество с каждым актером
actors_higher_mean = Counter()
for row in range(0, len(higher_mean)):
    next_row_data = higher_mean.iloc[row]
    for one_actor in next_row_data.cast.split('|'):
        actors_higher_mean[one_actor] += 1
        
# получаем максимальное значение счетчика
answers['16'] = actors_higher_mean.most_common(1)[0][0]
print(answers['16'])

Matt Damon


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [41]:
# +
# ВАРИАНТ 1
# получим счетчик жанров Николаса Кейджа
only_Cage_genres = actors_genres['Nicolas Cage']
# получим имя самого частого жанра
answers['17'] = only_Cage_genres.most_common(1)[0][0]
print(answers['17'])

Action


In [43]:
# +
# ВАРИАНТ 2, медленнее
genres_Cage = Counter()
for row in range(0, len(data)):
    next_row_data = data.iloc[row]
    if ('Nicolas Cage' in next_row_data.cast):
        for one_genre in next_row_data.genres.split('|'):
            genres_Cage[one_genre] += 1
            
genres_Cage.most_common(1)[0][0]

'Action'

In [44]:
# +
# ВАРИАНТ 2.5 - фильтрация фильмов без Николаса Кейджа вначале, потом тоже самое
movies_with_Cage = data[data.cast.str.contains('Nicolas Cage')]
genres_Cage2 = Counter()
for row in range(0, len(movies_with_Cage)):
    next_row_data = movies_with_Cage.iloc[row]
    for one_genre in next_row_data.genres.split('|'):
        genres_Cage2[one_genre] += 1
            
genres_Cage2.most_common(1)[0][0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [45]:
# +
# вначале получим фильмы Paramount Pictures
paramaunt_df = data[data.production_companies.str.contains('Paramount Pictures')]
# теперь выберем тот, что с минимальной прибылью т.е. максимальной убыточностью
answers['18'] = answer_string(paramaunt_df[paramaunt_df.profit == paramaunt_df.profit.min()])
print(answers['18'])

K-19: The Widowmaker (tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [46]:
# +
# Вариант 1
# сгруппируем по годам просуммировав сборы
revenue_grouped = data.groupby(['release_year']).sum().sort_values(by=['revenue'], ascending=False)
# получим самый успешный
answers['19'] = revenue_grouped.index[0]
print(answers['19'])

2015


In [48]:
# +
# Вариант 2 - простой, медленнее
revenues_sums = Counter()
for row in range(0, len(data)):
    next_row_data = data.iloc[row]
    revenues_sums[next_row_data.release_year] += next_row_data.revenue
revenues_sums.most_common(1)[0][0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [49]:
# +
# вначале получим фильмы Warner Bros
warner_df = data[data.production_companies.str.contains('Warner Bros')]
# сгруппируем по году и посчитаем сумму прибыли, отсортировав по убыванию
profits_by_year = warner_df.groupby(['release_year'])['profit'].sum().sort_values(ascending=False)
# получим самый прибыльный
answers['20'] = profits_by_year.index[0]
print(answers['20'])

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [50]:
# +
# посчитаем количества значений и получим аббревиатуру для самого частого
answers['21'] = calendar.month_abbr[data['release_month'].value_counts().index[0]]
### можно было бы без колонки, но получается громоздко
### answers['21'] = calendar.month_abbr[pd.DatetimeIndex(data['release_date']).month.value_counts().index[0]]
print(answers['21'])

Sep


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
# +
# создадим колонку сезона, month_to_season определена вначале файла
data['season'] = data.apply(lambda row: month_to_season(row['release_month']), axis=1)
### если бы не было колонки месяца можно было бы и без нее сразу получить, а так используем имеющееся
### data['season'] = data.apply(lambda row: month_to_season(row['release_date'].month), axis=1)

# подсчитаем значения и выберем лето
answers['22'] = data['season'].value_counts()['summer']
print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [52]:
# +
# выбираем зимние фильмы
data_winter = data[data.season == 'winter']
# выключаем предупреждение - здесь создаем колонку, никаких chained-операций
pd.options.mode.chained_assignment = None
# делаем новую колонку - массив директоров, а не строку
data_winter['array_directors'] = data_winter.apply(lambda row: row['director'].split('|'), axis=1)
# включаем предупреждение обратно
pd.options.mode.chained_assignment ='warn'
# размножаем/дублируем строки так, чтобы в каждой строке был один режиссер
data_winter_exploded = data_winter.explode('array_directors')
# выбираем самого продуктивного
answers['23'] = data_winter_exploded['array_directors'].value_counts(ascending=False).index[0]
print(answers['23'])

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [53]:
# +
# создадим выборку (сразу для двух вопросов)
df_title_overview_studios = data[['production_companies','original_title','overview']]
# выключаем предупреждение - здесь создаем колонки, никаких chained-операций
pd.options.mode.chained_assignment = None
# создаем столбцы длины названия в словах и длины описания в символах
df_title_overview_studios['word_title_count'] = df_title_overview_studios.apply(lambda row: len(row['original_title'].split(' ')), axis=1)
df_title_overview_studios['symbol_overview_count'] = df_title_overview_studios.apply(lambda row: len(row['overview']), axis=1)
# делаем новую колонку - массив директоров, а не строку
df_title_overview_studios['array_production_companies'] = df_title_overview_studios.apply(lambda row: row['production_companies'].split('|'), axis=1)
# включаем предупреждение обратно
pd.options.mode.chained_assignment ='warn'
# размножаем/дублируем строки так, чтобы в каждой строке была одна студия
df_exploded = df_title_overview_studios.explode('array_production_companies')
# группируем
grouped_by_studio = df_exploded.groupby('array_production_companies')
# агрегируем
aggregated = grouped_by_studio[['word_title_count', 'symbol_overview_count']].agg(['mean'])
# получем нужные студии - тут и в следующем вопросе
answers['24'] = aggregated[aggregated.word_title_count['mean'] == aggregated.word_title_count['mean'].max()].index[0]
print(answers['24'])

Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [54]:
# +
answers['25'] = aggregated[aggregated.symbol_overview_count['mean'] == aggregated.symbol_overview_count['mean'].max()].index[0]
print(answers['25'])

Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [55]:
# +
# не очень ясно в каком виде давать ответ, поэтому я введу переменные ответов и найду ту, 
# которая удовлетворяет 1% лучших
answer_variants = ['Inside Out, The Dark Knight, 12 Years a Slave', 'BloodRayne, The Adventures of Rocky & Bullwinkle',
   'Batman Begins, The Lord of the Rings: The Return of the King, Upside Down', '300, Lucky Number Slevin, Kill Bill: Vol. 1',
   'Upside Down, Inside Out, Iron Man']
# список листов ответов разделенных по фильмам
answer_variants_lists = []

# заполняем список листов ответов разделенных по фильмам
for i in range(0, len(answer_variants)):
    answer_variants_lists.append(answer_variants[i].split(', '))
#print(answer_variants_lists)

# получаем фильмы из квантиля и переводим в список
df_quantile_099 = data[data.vote_average >= data.quantile(0.99).vote_average]
df_quantile_099_list = df_quantile_099['original_title'].to_list()

# цикл идущий по спискам и проверяющий что фильмы из него в квантиле. Если нет, то сразу переходим к следующему
have_answer = False
for i in range(0, len(answer_variants_lists)):
    for j in range(0, len(answer_variants_lists[i])):
        if answer_variants_lists[i][j] not in df_quantile_099_list:
            break
    else:
        if (have_answer == True):
            print("Ошибка - два правильных ответа")
        else:
            have_answer = True
            answers['26'] = answer_variants[i]
print(answers['26'])

Inside Out, The Dark Knight, 12 Years a Slave


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [56]:
# +
# отберем одну колонку для удобства
df_cast = data[['cast']]
# выключаем предупреждение - здесь создаем колонки, никаких chained-операций
pd.options.mode.chained_assignment = None
# создаем новую колонку в которой массивы комбинаций пар актеров
df_cast['actors'] = df_cast.apply(lambda row: list(combinations(sorted(row['cast'].split('|')), 2)), axis=1)
# включаем предупреждение обратно
pd.options.mode.chained_assignment ='warn'
# размножаем/дублируем строки так, чтобы в каждой строке была одна студия
df_cast_ex = df_cast.explode('actors')
# группируем и аггрегируем
pairs_count = df_cast_ex.groupby(['actors']).agg(['count'])
pairs_count_max = pairs_count[pairs_count.cast['count'] == pairs_count.cast['count'].max()]
pairs_count_max.index.to_list()

[('Daniel Radcliffe', 'Emma Watson'),
 ('Daniel Radcliffe', 'Rupert Grint'),
 ('Emma Watson', 'Rupert Grint')]

ВАРИАНТ 2

In [57]:
# +
actors_value_counts = df_cast_ex['actors'].value_counts()
top_actors_pairs = actors_value_counts[actors_value_counts == actors_value_counts[0]]

In [58]:
# собственно получение ответа
test_answer_variants_27 = ['Johnny Depp & Helena Bonham Carter', 
                   'Ben Stiller & Owen Wilson',
                   'Daniel Radcliffe & Rupert Grint']
answers['27'] = check_correct_answer27(test_answer_variants_27, 
                                       [list(t) for t in pairs_count_max.cast.index.to_list()], 
                                       ' & ')
print(answers['27'])

Daniel Radcliffe & Rupert Grint


# Submission

In [59]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Sep',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [60]:
# убедиться что ничего не пропустил)
answers_len = len(answers)
# создаем ошибку если количество не равно 27 из теста =)
if (answers_len != 27):
    answers_len/0
answers_len

27

# Как указано тут https://skillfactorydspr.slack.com/archives/C017W2EQTRV/p1603282078051500?thread_ts=1603218900.048700&cid=C017W2EQTRV надо отдельно указать ответы в простом стиле, поэтому указываю

In [61]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
answers['2'] = 'Gods and Generals (tt0279111)'
answers['3'] = 'Winnie the Pooh (tt1449283)'
answers['4'] = 110
answers['5'] = 107
answers['6'] = 'Avatar (tt0499549)',
answers['7'] = 'The Lone Ranger (tt1210819)'
answers['8'] = 1478
answers['9'] = 'The Dark Knight (tt0468569)'
answers['10'] = 'The Lone Ranger (tt1210819)'
answers['11'] = 'Drama'
answers['12'] = 'Drama'
answers['13'] = 'Peter Jackson'
answers['14'] = 'Robert Rodriguez'
answers['15'] = 'Chris Hemsworth'
answers['16'] = 'Matt Damon'
answers['17'] = 'Action'
answers['18'] = 'K-19: The Widowmaker (tt0267626)'
answers['19'] = 2015
answers['21'] = 'Sep'
answers['20'] = 2014
answers['22'] = 450
answers['23'] = 'Peter Jackson'
answers['24'] = 'Four By Two Productions'
answers['25'] = 'Midnight Picture Show'
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

In [62]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': ('Avatar (tt0499549)',),
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Sep',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}